In [417]:
import numpy as np
import torch

In [418]:
torch.manual_seed(0)
from torchvision import transforms
import torch.nn.functional as F 
from torch import nn
import sys

sys.path.insert(1, '/Users/leonkiesgen/Documents/Python/BA_Optimization_ML/MaschineLearning/visualisation')
import json_ImExport
import LogDataImport

data=json_ImExport.loadData("/Users/leonkiesgen/Documents/Python/Kappa_variation_vmax100.json")

data2=LogDataImport.get_data("/Users/leonkiesgen/Documents/Python/mod_local_trajectory/logs/ltpl/2019_12_01/15_00_59_data.csv")


mean, std = (0.5), (0.5)

# Create a transform and normalise data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean, std)
                              ])




In [419]:
class FMNIST(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(99,300)
        self.fc2 = nn.Linear(300,200)
        self.fc3 = nn.Linear(200,150)
        self.fc4 = nn.Linear(150,98)

    def forward(self,x):

        x = F.leaky_relu(self.fc1(x),negative_slope=0.05)
        x = F.leaky_relu(self.fc2(x),negative_slope=0.05)
        x = F.leaky_relu(self.fc3(x),negative_slope=0.05)
        x = F.leaky_relu(self.fc4(x),negative_slope=0.05)
        #x= F.log_softmax(x_dim=1)

        return x

In [420]:
import random
model=FMNIST()

class Normalize():

    def __init__(self,data,scale=None):


        if scale is None:
            self.max=np.max(np.abs(data))
        else:
            self.max=scale
        print("Max is: ",self.max )

    def __call__(self,x):
        return x/self.max

    def normal(self,x):
        return x*self.max


def prepareData(data):
    
    prepareddata=[]
    for item in data:
        inpdata=[]
        #print(item)
        #print(item["v_ini"])
        inpdata.append(item["v_ini"])
        inpdata.append(item["v_max"])
        inpdata.append(item["Power"])   
        inpdata.extend(item["Kappa"])

        #print("Input: ",inpdata)
        label=item["V_op"]

        inpdata=torch.Tensor(inpdata)
        label=torch.Tensor(label)

        tupledata=(inpdata,label)
        prepareddata.append((inpdata,label))
    

    return prepareddata


def prepareLogData(data):
    lowestLen=200
    prepareddata=[]

    allkappa=[]
    for item in data:
        allkappa.extend(item[0])
    
    #Initialize Nomalizer
    kappaNormalize=Normalize(allkappa)
    VelNormalize=Normalize(None,scale=70)

    for item in data:
        inpdata=[]
        lowestLen= len(item[1]) if len(item[1])< lowestLen else lowestLen

        
        inpdata.append(VelNormalize(item[1][0]))#item["v_ini"])
        #print("Item 1: ", inpdata)
        #inpdata.append()#item["v_max"])
        #inpdata.append()#item["Power"]) 

        kappatmp=kappaNormalize(item[0][:98])
        inpdata.extend(kappatmp) #Kappa
        



        inpdata=torch.Tensor(inpdata)

        label=np.array(item[1][:98])

        label=torch.Tensor(VelNormalize(label)) #V_opt

        

        tupledata=(inpdata,label)
        prepareddata.append((inpdata,label))
    
    print(lowestLen)
    return prepareddata

traindata=prepareLogData(data2)
random.shuffle(traindata)
#print(traindata[1000][1])
#print(len(traindata))

Max is:  0.044784975560085524
Max is:  70
98


In [421]:
from IPython.core.debugger import set_trace

from torch import optim
criterion = nn.L1Loss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

OUTSHOW=None

num_epochs = 5000
train_tracker, test_tracker, accuracy_tracker = [], [], []


for i in range(num_epochs):
    cum_loss = 0
    cum_loss2 = 0
    random.shuffle(traindata)
    
    for batch, (inp, vel) in enumerate(traindata,1):


        optimizer.zero_grad()
        output = model(inp)

        

        loss = criterion(output, vel)

        loss.backward()
        optimizer.step()

        cum_loss += loss.item()
        

    train_tracker.append(cum_loss/len(traindata))
    print(f"Epoch({i+1}/{num_epochs}) | Training loss: {cum_loss2/len(traindata)} | " ,end="")
    
    test_loss = 0
    num_correct = 0
    total = 0
    
    for batch, (inp, vel) in enumerate(traindata,1):
        
        output3 = model(inp)
        batch_loss = criterion(output3, vel)        
        test_loss += batch_loss.item()
    
        
    
    test_tracker.append(test_loss/len(traindata))
    print(f"Test loss: {test_loss/len(traindata)} | ")
    '''
    #accuracy_tracker.append(num_correct/total)
    #print(f'Accuracy : {num_correct/total}')     
    

#print(f'\nNumber correct : {num_correct}, Total : {total}')
#print(f'Accuracy of the model after 30 epochs on the 10000 test images: {num_correct * 100 / total}% ')
    '''
    

Epoch(1/5000) | Training loss: 0.0 |Test loss: 0.2582786902269278 | 
Epoch(2/5000) | Training loss: 0.0 |Test loss: 0.14443341797028186 | 
Epoch(3/5000) | Training loss: 0.0 |Test loss: 0.08307227946829783 | 
Epoch(4/5000) | Training loss: 0.0 |Test loss: 0.05323184107032842 | 
Epoch(5/5000) | Training loss: 0.0 |Test loss: 0.03965592987564495 | 
Epoch(6/5000) | Training loss: 0.0 |Test loss: 0.033723036961537695 | 
Epoch(7/5000) | Training loss: 0.0 |Test loss: 0.028699603446024447 | 
Epoch(8/5000) | Training loss: 0.0 |Test loss: 0.024371479061116117 | 
Epoch(9/5000) | Training loss: 0.0 |Test loss: 0.02094544063607051 | 
Epoch(10/5000) | Training loss: 0.0 |Test loss: 0.018620134740507208 | 
Epoch(11/5000) | Training loss: 0.0 |Test loss: 0.016774284011685847 | 
Epoch(12/5000) | Training loss: 0.0 |Test loss: 0.01516265230383337 | 
Epoch(13/5000) | Training loss: 0.0 |Test loss: 0.013887061804585599 | 
Epoch(14/5000) | Training loss: 0.0 |Test loss: 0.01315752878181287 | 
Epoch(15/5

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(train_tracker, label='Training loss')
#plt.plot(test_tracker, label='Test loss')
plt.legend()

In [ ]:
def predict(testexample,NN=model):
    VelNormalize= Normalize(None,70)

    criterion = nn.L1Loss()
    pData=testexample[0]
    pred=NN(pData[0])
    loss=criterion(pred,pData[1])

    %matplotlib inline


    pred=VelNormalize.normal(np.array(pred.tolist()))
    real=VelNormalize.normal(np.array(pData[1]))

    plt.plot(pred,label="Pred")
    plt.plot(real,label= "Real_Optimizer")
    print("Loss:", loss.item())


def cumloss(example,NN=model):
    criterion = nn.L1Loss()
    cumloss=0
    i=0
    for (inp,label) in example:
        optimizer.zero_grad()
        pred=model.forward(inp)
        loss=criterion(pred,label)
        cumloss+= loss.item()
        i+=1
    return cumloss,i

In [ ]:

predict([traindata[35]],model)
#(cmloss,i)=cumloss(traindata)

#print("Cmloss: ",cmloss," Number ",i)

In [ ]:
%matplotlib inline

plt.plot(model.forward(traindata[1200][0]).tolist())
plt.plot(OUTSHOW.tolist(),label="Pred")
plt.plot(traindata[1200][1].tolist(),label= "Real_Optimizer")

In [ ]:
name="NN300200150_3"

In [ ]:
#Save MODEL
PATH="/Users/leonkiesgen/Documents/Python/BA_Optimization_ML/MaschineLearning/mlModels/"+ name
torch.save(model.state_dict(), PATH+".pt")
torch.save(model,PATH+".pth")

In [ ]:
#Load MODEL with same Structure
model=FMNIST()
model.load_state_dict(torch.load(PATH+"pt"))
model.eval()

In [ ]:
#Load MODEL with different Structure
model2=torch.load(PATH+".pth")
model2.eval()

In [ ]:
model2(traindata[37][0])